In [1]:
%%bash
chia keys derive -f 4001300365 wallet-address -n 5

Wallet address 0: txch1zgwteqg3gaqm88mu764xrqps8gh4lm5ape008xr2qg2u5345zwzswr3wvy
Wallet address 1: txch1va46l8pd2c2h2djs5jved3hfqkh0384tcda8347xt4u6rfea02lsztjjmv
Wallet address 2: txch15c59kptqzxj9s0jqwm9035pkldg3qcy42kl7czsu6wyp465twwlsjcdehn
Wallet address 3: txch1tngsu0rj98c2c077rdkdhxt9pljtzg9mq2rxy3jwl769ltxe43lqskcu9h
Wallet address 4: txch1dagw2tqqfzzfy0krhqvp9dlgrmr7e774cjlfjlutyqtkas67j44sxu3afy


In [2]:
%%bash
chia wallet show -f 4001300365

Wallet height: 11
Sync status: Synced
Balances, fingerprint: 4001300365

Chia Wallet:
   -Total Balance:         1.999499999997 txch (1999499999997 mojo)
   -Pending Total Balance: 1.999499999997 txch (1999499999997 mojo)
   -Spendable:             1.999499999997 txch (1999499999997 mojo)
   -Type:                  STANDARD_WALLET
   -Wallet ID:             1

Profile 1:
   -Total Balance:         1.0  
   -Pending Total Balance: 1.0  
   -Spendable:             1.0  
   -Type:                  DECENTRALIZED_ID
   -DID ID:                did:chia:1fcs44905vc749sdcnqem4xh3fzmmx65p7htrz4p2rnrhe9m4jvgqy5j9pa
   -Wallet ID:             2

Profile 2:
   -Total Balance:         1.0  
   -Pending Total Balance: 1.0  
   -Spendable:             1.0  
   -Type:                  DECENTRALIZED_ID
   -DID ID:                did:chia:1u7mstlcyenx22jnzyucnhzrnzecg0gr7996n0alkncl09gdnd0pq0fz0c8
   -Wallet ID:             4

NFT Wallet:
   -Total Balance:         0.0  
   -Pending Total Balance: 0.0  

In [12]:
import aiosqlite
from pathlib import Path
DB_VERSION = 2
#DB_PATH = Path(f'/Users/karlkim/.chia/simulator/hints/db/blockchain_v2_simulator0.sqlite')
DB_PATH = Path('/Users/karlkim/.chia/simulator/pretty-chia/db/blockchain_v2_simulator0.sqlite')

In [13]:
from chia.consensus.block_record import BlockRecord
query = f"""SELECT coin_id, hint from hints"""
async with aiosqlite.connect(DB_PATH) as db:
    async with db.execute(query) as cursor:
        async for row in cursor:
            coin_id = row[0]
            hint = row[1]
            addr = encode_puzzle_hash(hint, 'txch')
            print(f'coin_id: {coin_id.hex()}')
            print(f'hint:\t {hint.hex()}')
            print(f'addr:\t {addr}\n')

coin_id: 596f6ccc43b767a6dbbf8973914d5fb067923d7fd2d64255f461f87c0ecce38c

hint:    078662243bf9d36adaa526cb34bf1592dbef29438ce675bc2a822bd726dfb323

addr:    txch1q7rxyfpml8fk4k49ym9nf0c4jtd7722r3nn8t0p2sg4awfklkv3s7lj6vj

coin_id: 21682522f8cc7be356f030f5d973a8c2515c3287927c125b17182e39faf81f53

hint:    9372962908dcdeb01ae5cf06b086272c3c4193b984602a5b0d319b5b0862d3a5

...

hint:    c0f3487b205d3e0bb545513bd54ea0486c47206c1eeec5af6e0bea977d1c048e

addr:    txch1cre5s7eqt5lqhd292yaa2n4qfpkywgrvrmhvttmwp04fwlguqj8qhrvlls

coin_id: 574fd01374f605006065d9426d0c828cae4ba7cb0f7a226de0ed8d084773077f

hint:    c0f3487b205d3e0bb545513bd54ea0486c47206c1eeec5af6e0bea977d1c048e

addr:    txch1cre5s7eqt5lqhd292yaa2n4qfpkywgrvrmhvttmwp04fwlguqj8qhrvlls

In [15]:
from chia.types.blockchain_format.sized_bytes import bytes32
from chia.types.coin_record import CoinRecord
from chia.util.ints import uint32, uint64
def get_coin_record(row):
    return CoinRecord(
        Coin(row[4], row[3], uint64.from_bytes(row[5])), 
        row[0], row[1], row[2], row[6]
    ) 
hint = "0xc0f3487b205d3e0bb545513bd54ea0486c47206c1eeec5af6e0bea977d1c048e"
async with aiosqlite.connect(DB_PATH) as db:
    query = f"""SELECT confirmed_index, spent_index, coinbase, puzzle_hash, 
        coin_parent, amount, timestamp 
        FROM coin_record
        JOIN hints h
        WHERE coin_name=h.coin_id
            AND h.hint=?
        ORDER BY spent_index, confirmed_index
    """
 
    async with db.execute(query, (bytes32.from_hexstr(hint) ,)) as cursor:
        async for row in cursor:
            coin_record = get_coin_record(row)
            print(coin_record.coin.name().hex())

574fd01374f605006065d9426d0c828cae4ba7cb0f7a226de0ed8d084773077f

f7f8a57086b0bdf516016ddd3b9f4a92d7e8949f052c46aa19390c9d509aa41d